<a href="https://colab.research.google.com/github/justcating/pypairs/blob/main/%D0%94%D0%97_12.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Імпорт модулів

In [1]:
import pandas as pd
import torch

Дані медичне страхування

In [2]:
# трохи обробки даних
df = pd.read_csv('https://raw.githubusercontent.com/HalyshAnton/IT-Step-Pyton-AI/main/module3/data/insurance.csv')

df['sex'] = df['sex'].map({'male': 0, 'female':1})
df['smoker'] = df['smoker'].map({'no': 0, 'yes':1})

df = df.drop(columns=['region'])

data = df[['age', 'bmi', 'expenses']]
df[['age', 'bmi', 'expenses']] = (data - data.min()) / (data.max() - data.min())

In [3]:
df.head()

,age,sex,bmi,children,smoker,expenses
0,0.021739,1,0.320755,0,1,0.251611
1,0.000000,0,0.479784,1,0,0.009636
2,0.217391,0,0.458221,3,0,0.053115
3,0.326087,0,0.180593,0,0,0.333010
4,0.304348,0,0.347709,0,0,0.043816


Потрібно на основі інформації людини спрогнозувати розмір її страховки `expenses`

# Завдання 1
Створіть клас Dataset

In [4]:
from torch.utils.data import Dataset

class MyDataset(Dataset):
  def __init__(self, X, y):
    super().__init__()
    self.X = torch.tensor(X.values, dtype=torch.float32)
    self.y = torch.tensor(y.values, dtype=torch.float32)
    self.y = self.y.reshape(-1,1)

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, ind):
    return self.X[ind], self.y[ind]

X = df.drop(columns="expenses")
y = df['expenses']

data = MyDataset(X, y)

# Завдання 2
Створіть DataLoader. Перевірте розміри даних в першому пакеті. Розмір пакетів оберіть самостійно

In [5]:
from torch.utils.data import DataLoader

loader = DataLoader(data, batch_size=32, shuffle=True)

first_batch = next(iter(loader))

X_batch, y_batch = first_batch

X_batch.shape
y_batch.shape

torch.Size([32, 1])

# Завдання 3
Створіть нейромережу:
* Кількість шарів оберіть між 1-5
* Виберіть різні функції активації: Relu, LeakyRelu
* Кілкість нейронів у прихованих шарах повинна **не зростати**

In [6]:
from torch import nn

model = nn.Sequential(
    nn.Linear(5,5),
    nn.ReLU(),
    nn.Linear(5,3),
    nn.LeakyReLU(),
    nn.Linear(3,1)
)

model

Sequential(
  (0): Linear(in_features=5, out_features=5, bias=True)
  (1): ReLU()
  (2): Linear(in_features=5, out_features=3, bias=True)
  (3): LeakyReLU(negative_slope=0.01)
  (4): Linear(in_features=3, out_features=1, bias=True)
)

# Завдання 4
Визначте функцію втрат та оптимізатор

In [8]:
loss_fn = nn.MSELoss()

optimizer = torch.optim.Adam(model.parameters(), lr=.005)

# Завдання 5
Натренуйте нейромережу. візміть невелику кількість епох

In [9]:
epoches = 20

for i in range(epoches):
  for X, y in loader:

    y_pred = model(X)

    loss = loss_fn(y_pred, y)

    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

  print(loss.item())

0.026038317009806633
0.02426234818994999
0.005108150653541088
0.00381866074167192
0.002422603312879801
0.010809233412146568
0.008914283476769924
0.017910590395331383
0.0038041346706449986
0.005169228184968233
0.012025192379951477
0.007287316489964724
0.01402988564223051
0.01437520980834961
0.0023967684246599674
0.006330440752208233
0.0036501502618193626
0.002745632315054536
0.009955406188964844
0.0036042530555278063
